## Word Anology Preprocessing

In [2]:
import os
import collections
import smart_open
import random
import numpy as np

def load_ingredients(path):
    ingredients = {}
    ingredients_list = []
    with open(path, 'r') as f:
        for line in f:
            if line[0] == '#':
                pass
            else:
                line_split = line.rstrip().split('\t')
                ingredients_id = line_split[0]
                ingredients_list = line_split[1:]
                ingredients[ingredients_id] = ingredients_list
    return ingredients

In [50]:
path_data = '..' + os.sep + 'data'
path_ingr_info = path_data + os.sep + 'scientific_report' + os.sep + 'ingr_info.tsv'
ingredients = load_ingredients(path_ingr_info)

In [51]:
ingredients_list = []
for ingr_id in ingredients:
    ingredients_list.append(ingredients[ingr_id][0])

In [52]:
target_ingr_list = ["japanese","chinese","california","thai","spanish","asian","ethiopian","american","european"]


for target_ingr in target_ingr_list:
    #print target_ingr
    
    target_list = []
    # all ingredients to extract target ingredients
    for ingr in ingredients_list:
        if target_ingr in ingr:
            target_list.append(ingr)
    #print target_list

    # target ingrdients
    for target in target_list:
        target_split = target.split("_")
        try:
            target_split.remove(target_ingr)
        except ValueError :  # 에러 종류
            continue

        target_combined = "_".join(target_split)

        if target_combined in ingredients_list:
            print "%s\t%s" % (target_combined, target)

star_anise_oil	japanese_star_anise_oil
peppermint_oil	japanese_peppermint_oil
mint	japanese_mint
hop	japanese_hop
whiskey	japanese_whiskey
star_anise	japanese_star_anise
peppermint	japanese_peppermint
calamus	japanese_calamus
plum	japanese_plum
hop_oil	japanese_hop_oil
cabbage	chinese_cabbage
star_anise_oil	chinese_star_anise_oil
quince	chinese_quince
cinnamon	chinese_cinnamon
star_anise	chinese_star_anise
orange_peel	california_orange_peel
orange	california_orange
lemon_oil	california_lemon_oil
pepper	california_pepper
pepper	thai_pepper
origanum	spanish_origanum
anise	spanish_anise
sage	spanish_sage
origanum_oil	spanish_origanum_oil
roasted_peanut	roasted_spanish_peanut
storax	asian_storax
pepper	ethiopian_pepper
storax	american_storax
peppermint	american_peppermint
pennyroyal	american_pennyroyal
peppermint_oil	american_peppermint_oil
potato_chip	american_potato_chip
cranberry	american_cranberry
cranberry	european_cranberry


## Anology Test

In [76]:
import os
import collections
import smart_open
import random
import numpy as np

from utils import DataLoader, GensimModels, DataPlotter
gensimLoader = GensimModels.GensimModels()
path_results = ".." + os.sep + "results"
path_embeddings_ingredients = path_results + os.sep + 'embeddings' + os.sep + "embeddings_ingredients_f-5_rs-True_ns-50_charemb-True_dim50.bin"
model = gensimLoader.load_word2vec(path=path_embeddings_ingredients)

# model.wv.most_similar(positive=['woman', 'king'], negative=['man'])
# orange + citrus_oil - citrus_flower_oil = orange_flower


In [101]:
path_data = '..' + os.sep + 'data'
path_ingr_anol = path_data + os.sep + 'ingredient-anology-4.txt'

anology_dict = {}
anology_list = []

f_origin = open(path_ingr_anol, 'r')
#whole file
for line_origin in f_origin:
    if line_origin[0] == ':':
        continue
    #whole file 2
    f_target = open(path_ingr_anol, 'r')
    for line_target in f_target:
        if line_target[0] == ":":
            continue
            
        line_origin_split = line_origin.rstrip().split("\t")
        line_target_split = line_target.rstrip().split("\t")

        if line_origin_split == line_target_split:
            continue
        
        anology_list.append(line_origin_split + line_target_split)
anology_list    

[['rose', 'rose_wine', 'strawberry', 'strawberry_wine'],
 ['rose', 'rose_wine', 'bilberry', 'bilberry_wine'],
 ['rose', 'rose_wine', 'passion_fruit', 'passion_fruit_wine'],
 ['rose', 'rose_wine', 'plum', 'plum_wine'],
 ['rose', 'rose_wine', 'sherry', 'sherry_wine'],
 ['rose', 'rose_wine', 'scotch', 'scotch_whiskey'],
 ['rose', 'rose_wine', 'malt', 'malt_whiskey'],
 ['rose', 'rose_wine', 'pear', 'pear_brandy'],
 ['rose', 'rose_wine', 'fruit', 'fruit_brandy'],
 ['rose', 'rose_wine', 'grape', 'grape_brandy'],
 ['rose', 'rose_wine', 'anise', 'anise_brandy'],
 ['rose', 'rose_wine', 'cherry', 'cherry_brandy'],
 ['rose', 'rose_wine', 'blackberry', 'blackberry_brandy'],
 ['rose', 'rose_wine', 'rum', 'rum_brandy'],
 ['rose', 'rose_wine', 'plum', 'plum_brandy'],
 ['rose', 'rose_wine', 'apple', 'apple_brandy'],
 ['rose', 'rose_wine', 'raspberry', 'raspberry_brandy'],
 ['rose', 'rose_wine', 'brandy', 'rum_brandy'],
 ['rose', 'rose_wine', 'brandy', 'cherry_brandy'],
 ['strawberry', 'strawberry_wine

In [102]:
def analogy_test(word_vectors):
    top1 = 0
    top5 = 0
    top10 = 0
    top50 = 0

    for anology in anology_list:
        word1 = anology[0] # positive
        word1_1 = anology[1] # answer
        word2 = anology[2] # negative
        word2_1 = anology[3] # positive

        try:
            list_most_similar = word_vectors.most_similar(positive=[word1, word2_1], negative=[word2], topn = 50)
            list_most_similar_cosmul = word_vectors.most_similar_cosmul(positive=[word1, word2_1], negative=[word2], topn = 30)

        except KeyError as ke:
            #print ke
            continue
            
        #print "\nIngredient Analogy Test"
        #print "%s + %s - %s = %s" % (word1, word2_1, word2, word1_1)
        for ix, dic in enumerate(list_most_similar):
            word = dic[0]
            score = dic[1]

            if word1_1 == word:
                if ix+1 == 1:
                    top1 += 1
                if ix+1 <= 5:
                    top5 += 1
                if ix+1 <= 10:
                    top10 += 1
                if ix+1 <= 50:
                    top50 += 1
                
                print "\nIngredient Analogy Test"
                print "%s + %s - %s = %s" % (word1, word2_1, word2, word1_1)
                print word, score, ix+1

    num_test_sets = len(anology_list)
    print "\n\nTotal Number of Test Sets:", num_test_sets
    print "Top1 Accuracy: %d, %f%%" % (top1, top1/float(num_test_sets)*100)
    print "Top5 Accuracy: %d, %f%%" % (top5, top5/float(num_test_sets)*100)
    print "Top10 Accuracy: %d, %f%%" % (top10, top10/float(num_test_sets)*100)
    print "Top50 Accuracy: %d, %f%%" % (top50, top50/float(num_test_sets)*100)

analogy_test(model)


Ingredient Analogy Test
rose + bilberry_wine - bilberry = rose_wine
rose_wine 0.523407459259 2

Ingredient Analogy Test
rose + fruit_brandy - fruit = rose_wine
rose_wine 0.40338036418 17

Ingredient Analogy Test
strawberry + rose_wine - rose = strawberry_wine
strawberry_wine 0.502189218998 29

Ingredient Analogy Test
strawberry + apple_brandy - apple = strawberry_wine
strawberry_wine 0.558084309101 9

Ingredient Analogy Test
malt + grape_brandy - grape = malt_whiskey
malt_whiskey 0.332618057728 47

Ingredient Analogy Test
pear + bilberry_wine - bilberry = pear_brandy
pear_brandy 0.455010563135 26

Ingredient Analogy Test
pear + malt_whiskey - malt = pear_brandy
pear_brandy 0.432617366314 48

Ingredient Analogy Test
pear + grape_brandy - grape = pear_brandy
pear_brandy 0.364712834358 42

Ingredient Analogy Test
plum + rose_wine - rose = plum_brandy
plum_brandy 0.505302786827 15

Ingredient Analogy Test
plum + bilberry_wine - bilberry = plum_brandy
plum_brandy 0.539036750793 5

Ingredie